# Quick projections of 2-channel 3D images (```.msr``` Version) 

## Input

This recipe expects an input folder containing 3D & 2 channel ```.msr``` files.


## Output

For each ```.msr``` image in the input folder, a ```.png``` image showning z-,y- and x-projections, with the color channels merged will be created in the output folder. The name of output files will be ```{input file name (without ending)}_projected.png```

In [ ]:
import numpy as np
import warnings
from pathlib import Path
from skimage.io import imsave
from utils.projection import get_orthogonal_projections_8bit
from msr_reader import OBFFile
import matplotlib.pyplot as plt
from xml.etree import ElementTree

def load_msr(file):
    with OBFFile(file) as f:
            
        imgs = []
        channel_names = []

        for idx in range(0,len(f.shapes)):

            # reading image data
            img = f.read_stack(idx) # read stack with index idx into numpy array
            imgs.append(img)

            # metadata
            stack_sizes = f.shapes # list of stack sizes/shapes, including stack and dimension names
            pixel_sizes = f.pixel_sizes # like sizes, but with pixel sizes (unit: meters)
            channel_names.append(pixel_sizes[idx].name)
            # pixel_sizes = [pixel_sizes[idx].sizes['ExpControl Z'],
            #                pixel_sizes[idx].sizes['ExpControl Y'],
            #                pixel_sizes[idx].sizes['ExpControl X']]
            pixel_sizes = [pixel_sizes[idx].sizes[0],
                   pixel_sizes[idx].sizes[1],
                   pixel_sizes[idx].sizes[2]]

        imgs = np.asarray(imgs)

        return(imgs, channel_names, pixel_sizes)

In [ ]:
# path containing files to visualize (upper level directory containing "raw" directory)
in_path = "/home/stumberger/ep2024/example2/"

# default: put results in subdirectory called 'projections'
out_subdirectory = 'projections'

# Intensity range for levelling
# Default: auto -> level based on quantiles 
intensity_range = 'auto'
auto_range_quantiles = (0.02, 0.9995)

# Alternatively, specify range directly
# intensity_range = (500, 1500)

# Type of projection, can be min/max/mean
projection_type='max'

In [ ]:
# complete in and out paths
path = Path(in_path) / 'raw'
out_path = Path(in_path) / out_subdirectory

# get all msr files in in_path
in_files = sorted(list(Path(path).glob('*.msr')))

# show for verification
in_files

In [ ]:
# make folder for results if it does not exist
if not out_path.exists():
    out_path.mkdir(parents=True)

for in_file in in_files:

    # read images, channel names, pixel sizes
    try:
        imgs, channel_names, pixel_size = load_msr(in_file)
    except:
        print(f'error loading file {in_file}, skipping')
        continue

    projs = []
    # do projections and save
    for img, channel_name in zip(imgs, channel_names):

        projections_8bit = get_orthogonal_projections_8bit(img, pixel_size,
                projection_type=projection_type, intensity_range=intensity_range, auto_range_quantiles=auto_range_quantiles)

        projs.append(projections_8bit)
        
        
    # make filepath for output
    outfile = out_path / (in_file.stem + f'_projected.png')
    
    merged_img = np.dstack((projs[0],projs[1],np.zeros_like(projs[0])))

    # catch low contrast warning
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)

        imsave(str(outfile), merged_img)
    
    print(f'saved projections of {str(in_file)}.')